In [1]:
import os
import pandas as pd

In [2]:
source_folder = os.getcwd() + "\\CodeSnippets\\Source"
files = os.listdir(source_folder)
categorie = ["HOF", "I", "LC", "R"]

df = pd.DataFrame([], columns=["Name", "Code", "Categorie"])
for idx, file in enumerate(files):
    filename, _file_extension = os.path.splitext(file)
    file_data = open(source_folder + "\\" + file, "r")
    file_data = file_data.read()
    df = df.append(pd.DataFrame([[filename, file_data, categorie[idx%4]]], columns=["Name", "Code", "Categorie"]))
    
df = df.reset_index()
df = df.drop(["index"], axis=1)
df["LOC"] = df.apply(lambda row: row["Code"].count("\n"), axis=1)
df["CPS"] = df.apply(lambda row: len(row["Code"]), axis=1)
df["LOC/CPS"] = df.apply(lambda row: row["CPS"]/row["LOC"], axis=1)

In [3]:
cols = ["Categorie", "LOC-mean", "LOC-std", "CPS-mean", "LOC-std", "LOC/CPS-mean", "LOC/CPS-std"]
df_confounding = pd.DataFrame([], columns=cols)
for cat in categorie:
    data = df.loc[df["Categorie"]==cat].describe()
    df_confounding = df_confounding.append(pd.DataFrame([[cat,
                                                          data["LOC"]["mean"],
                                                          data["LOC"]["std"],
                                                          data["CPS"]["mean"],
                                                          data["CPS"]["std"],
                                                          data["LOC/CPS"]["mean"],
                                                          data["LOC/CPS"]["std"]]], columns=cols))
df_confounding.to_excel("CodeSnippets/confoudning.xlsx", index=False)